In [1]:
import keras
import tensorflow as tf

print(keras.__version__)
print(tf.__version__)


3.3.2
2.16.1


In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, concatenate
from tensorflow.keras.applications import EfficientNetB4

def build_model_with_saliency(input_shape=(380, 380, 6), num_classes=6):
    base_model = EfficientNetB4(include_top=False, weights=None, input_shape=(380, 380, 3))

    # Split the input into image and saliency map
    inputs = Input(input_shape)
    image_input = inputs[:, :, :, :3]
    saliency_input = inputs[:, :, :, 3:]

    # Process the image input
    x = base_model(image_input, training=False)
    x = GlobalAveragePooling2D()(x)

    # Process the saliency map
    y = base_model(saliency_input, training=False)
    y = GlobalAveragePooling2D()(y)

    # Combine both processed inputs
    combined = concatenate([x, y])
    combined = Dense(256, activation='relu')(combined)
    combined = Dropout(0.5)(combined)
    output = Dense(num_classes, activation='softmax')(combined)

    model = Model(inputs=inputs, outputs=output)
    return model

model_with_saliency = build_model_with_saliency(num_classes=6)
model_with_saliency.load_weights('best_EfficientNetB4_weights.weights.h5')

In [ ]:
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg16
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50
from tensorflow.keras.applications import EfficientNetB4, InceptionResNetV2
import matplotlib.pyplot as plt


try:
    model = load_model('best_ResNet50_model.keras')
except Exception as e:
    print(f"Error loading model: {e}")
    # Attempt to rebuild the model architecture and load weights separately
    model = build_model_with_saliency(num_classes=num_classes)
    model.load_weights('best_ResNet50_model.keras')

In [1]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg16
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50
from tensorflow.keras.applications import EfficientNetB4, InceptionResNetV2
import matplotlib.pyplot as plt


model_skin_cnn = load_model('CNN_skin_classifier_weights.weights.h5')
# model_derm_vgg16 = load_model('OneDrive - Solent University/Diss/streamlit/Demascopy_models/vgg16_model_with_saliency.h5')
# model_derm_resnet50 = load_model('OneDrive - Solent University/Diss/streamlit/Demascopy_models/best_ResNet50_model.h5')
# model_derm_efficientnet = load_model('OneDrive - Solent University/Diss/streamlit/Demascopy_models/efficientnetb4_model_with_saliency.h5')
# model_derm_inceptionresnetv2 = load_model('OneDrive - Solent University/Diss/streamlit/Demascopy_models/InceptionResNetV2_model.h5')

ValueError: No model config found in the file at CNN_skin_classifier_weights.weights.h5.

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2

def build_model(input_shape=(224, 224, 6)):  # 6 channels for image + saliency
    inputs = Input(input_shape)
    image_input = inputs[:, :, :, :3]
    saliency_input = inputs[:, :, :, 3:]

    def create_cnn(input_tensor):
        conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(input_tensor)
        conv1 = BatchNormalization()(conv1)
        conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(conv1)
        conv1 = BatchNormalization()(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        pool1 = Dropout(0.5)(pool1)
        conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(pool1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(conv2)
        conv2 = BatchNormalization()(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        pool2 = Dropout(0.5)(pool2)
        conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(pool2)
        conv3 = BatchNormalization()(conv3)
        conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(conv3)
        conv3 = BatchNormalization()(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        pool3 = Dropout(0.5)(pool3)
        conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(pool3)
        conv4 = BatchNormalization()(conv4)
        conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(conv4)
        conv4 = BatchNormalization()(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
        pool4 = Dropout(0.5)(pool4)
        conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(pool4)
        conv5 = BatchNormalization()(conv5)
        conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(conv5)
        conv5 = BatchNormalization()(conv5)
        pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)
        pool5 = Dropout(0.5)(pool5)
        flatten = Flatten()(pool5)
        return flatten

    image_features = create_cnn(image_input)
    saliency_features = create_cnn(saliency_input)
    concatenated_features = tf.keras.layers.Concatenate()([image_features, saliency_features])
    dense1 = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(concatenated_features)
    dense1 = Dropout(0.5)(dense1)
    output = Dense(6, activation='softmax')(dense1)

    model = Model(inputs=inputs, outputs=output)
    return model

# Create the model
model_with_saliency = build_model()

# Attempt to load weights
weights_path = 'CNN_skin_classifier_weights.weights.h5'
try:
    model_with_saliency.load_weights(weights_path)
    print("Weights loaded successfully.")
except ValueError as e:
    print(f"Error loading weights: {e}")

# Print the model summary to verify
model_with_saliency.summary()


Weights loaded successfully.


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)    │ (None, 224, 224, 6)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item_4 (GetItem)          │ (None, 224, 224, 3)       │               0 │ input_layer_8[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item_5 (GetItem)          │ (None, 224, 224, 3)       │               0 │ input_layer_8[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_40 (Conv2D)            │ (None, 224, 224, 32)      │             896 │ get_item_4[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_50 (Conv2D)            │ (None, 224, 224, 32)      │             896 │ get_item_5[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_40        │ (None, 224, 224, 32)      │             128 │ conv2d_40[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_50        │ (None, 224, 224, 32)      │             128 │ conv2d_50[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_41 (Conv2D)            │ (None, 224, 224, 32)      │           9,248 │ batch_normalization_40[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_51 (Conv2D)            │ (None, 224, 224, 32)      │           9,248 │ batch_normalization_50[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_41        │ (None, 224, 224, 32)      │             128 │ conv2d_41[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_51        │ (None, 224, 224, 32)      │             128 │ conv2d_51[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_20              │ (None, 112, 112, 32)      │               0 │ batch_normalization_41[0]… │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_25              │ (None, 112, 112, 32)      │               0 │ batch_normalization_51[0]… │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_25 (Dropout)          │ (None, 112, 112, 32)      │               0 │ max_pooling2d_20[0][0]     │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 22,287,174 (85.02 MB)

 Trainable params: 22,279,238 (84.99 MB)

 Non-trainable params: 7,936 (31.00 KB)

In [7]:
from tensorflow.keras.models import load_model
import joblib

# Load the model
try:
    model = load_model('best_EfficientNetB4_model')
except Exception as e:
    print(f"Error loading SavedModel format: {e}")
    
    # Fallback: Load model architecture and weights separately
    try:
        from tensorflow.keras.models import model_from_json
        
        # Load model architecture
        with open("best_EfficientNetB4_model_architecture.json", "r") as json_file:
            loaded_model_json = json_file.read()
        model = model_from_json(loaded_model_json)
        
        # Load model weights
        model.load_weights("best_EfficientNetB4_model_weights.h5")
        
        print("Model loaded from architecture and weights files.")
    except Exception as e2:
        print(f"Error in fallback loading method: {e2}")

# Load the label encoder
label_encoder = joblib.load('label_encoder.joblib')

Error loading SavedModel format: File format not supported: filepath=best_EfficientNetB4_model. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(best_EfficientNetB4_model, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).
Error in fallback loading method: [Errno 2] No such file or directory: 'best_EfficientNetB4_model_architecture.json'


C:\Users\adedi\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
import tensorflow as tf
from tensorflow import keras
import joblib
import warnings

# Load the model
try:
    # Try loading as a Keras 3 compatible .keras file
    model = keras.models.load_model('best_EfficientNetB4_model.keras')
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading .keras format: {e}")
    
    try:
        # Try loading as a TensorFlow SavedModel
        model = keras.layers.TFSMLayer('best_EfficientNetB4_model', call_endpoint='serving_default')
        print("Model loaded as TFSMLayer.")
    except Exception as e2:
        print(f"Error loading as TFSMLayer: {e2}")
        
        try:
            # Try loading as H5 format
            model = keras.models.load_model('best_EfficientNetB4_model.h5')
            print("Model loaded from H5 format.")
        except Exception as e3:
            print(f"Error loading H5 format: {e3}")
            print("Unable to load the model in any format.")

# Load the label encoder with a warning filter
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    try:
        label_encoder = joblib.load('label_encoder.joblib')
        print("Label encoder loaded successfully.")
    except Exception as e:
        print(f"Error loading label encoder: {e}")

# Optionally, load the training history if you saved it
import json
try:
    with open('training_history.json', 'r') as f:
        history = json.load(f)
    print("Training history loaded successfully.")
except Exception as e:
    print(f"Error loading training history: {e}")

Error loading .keras format: Could not deserialize class 'Functional' because its parent module keras.src.engine.functional cannot be imported. Full object config: {'module': 'keras.src.engine.functional', 'class_name': 'Functional', 'config': {'name': 'model_1', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 380, 380, 6], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_4'}, 'registered_name': None, 'name': 'input_4', 'inbound_nodes': []}, {'module': 'keras.src.layers.core.tf_op_layer', 'class_name': 'SlicingOpLambda', 'config': {'name': 'tf.__operators__.getitem', 'trainable': True, 'dtype': 'float32', 'function': '__operators__.getitem'}, 'registered_name': 'SlicingOpLambda', 'build_config': {'input_shape': [None, 380, 380, 6]}, 'name': 'tf.__operators__.getitem', 'inbound_nodes': [['input_4', 0, 0, {'slice_spec': [{'start': None, 'stop': None, 'step': None}, {'start': None, 'stop': None